In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn import functional as F
from torch import nn, optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.distributions.multivariate_normal import MultivariateNormal
from sklearn.datasets import make_sparse_spd_matrix#生成稀疏正定对称精度矩阵
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
class custom_graphconv(nn.Module):
    def __init__(self,in_channels,out_channels,max_kernel_size,feature):
        super(custom_graphconv,self).__init__()
        self.in_channels=in_channels
        self.out_channels=out_channels
        self.max_kernel_size=max_kernel_size
        self.conv_array=nn.Parameter(torch.randn(out_channels,in_channels,max_kernel_size))#核参数数组
        self.bias=nn.Parameter(torch.randn(feature,out_channels))#偏置
        self.feature=feature# the dimensionality of variable
        self.kernel_array=torch.zeros(self.in_channels,self.feature*self.feature,self.feature,out_channels).to(device)
        #kernel size-> in_channel*feature*feature*out_channel
    def convert_to_kernel(self,adjacent,adj2):#adjacent with diagonal
        conv_array2=torch.t(self.conv_array.reshape(self.out_channels*self.in_channels,self.max_kernel_size))
        
        temp_array=torch.mm(adj2,conv_array2)
        temp_array=torch.transpose(temp_array,1,0)
        temp_array=temp_array.reshape(1,-1).reshape(conv_array2.shape[1]*adj2.shape[1],adj2.shape[1]).reshape(conv_array2.shape[1],adj2.shape[1],adj2.shape[1])
        #cc=cc.reshape((1,conv_array2.shape[1]*adj2.shape[0]),'F').reshape(conv_array2.shape[1]*adj2.shape[1],adj2.shape[1]).reshape(conv_array2.shape[1],adj2.shape[1],adj2.shape[1])
        temp_array=temp_array.reshape(self.out_channels,self.in_channels,self.feature,self.feature)
        temp_array=temp_array.permute((1,0,2,3))
        temp_array=temp_array.permute((0,2,3,1))
        
        return temp_array
    def forward(self,x,adjacent,adj2):
        self.kernel_array=self.convert_to_kernel(adjacent,adj2)
        x=x.view(-1,self.feature*self.in_channels)#input x of size -> Batch*feature*in_channel
        kernel_array=self.kernel_array.contiguous().view(self.in_channels*self.feature,self.feature*self.out_channels)
        output=torch.mm(x,kernel_array)
        return output.view(-1,self.feature,self.out_channels)+self.bias#output size -> Batch*feature*out_channel

In [ ]:
class GCN_model(nn.Module):
    def __init__(self,feature,max_kernel_size,in_channel1,out_channel1,out_channel2,out_channel3,hidden1,out):
        super(GCN_model,self).__init__()
        self.conv1=custom_graphconv(in_channel1,out_channel1,max_kernel_size,feature)
        self.conv2=custom_graphconv(out_channel1,out_channel2,max_kernel_size,feature)
        self.conv3=custom_graphconv(out_channel2,out_channel3,max_kernel_size,feature)
        self.dense1=nn.Linear(feature,hidden1)
        self.dense2=nn.Linear(hidden1,out)
    def forward(self,x,adjacent,adj2):
        con_out=self.conv1(x,adjacent,adj2)
        con_out=self.conv2(con_out,adjacent,adj2)
        con_out=self.conv3(con_out,adjacent,adj2)
        out=torch.sigmoid(self.dense1(torch.mean(con_out,dim=-1)))
        out=torch.sigmoid(self.dense2(out))
        return out

In [ ]:
class custom_data(torch.utils.data.dataset.Dataset):
    def __init__(self,mu,precision,nsample,label_index):
        self.m=MultivariateNormal(loc=mu,precision_matrix=precision)
        self.data=self.m.rsample(torch.Size([nsample]))
       
        self.n=nsample
        self.index=label_index
    def __getitem__(self,index):
        data=self.data
        data=(self.data-torch.min(self.data,0)[0])/(torch.max(self.data,0)[0]-torch.min(self.data,0)[0])#归一化
        mean=(torch.max(data[:,self.index])+torch.min(data[:,self.index]))/2
        label=(data[:,self.index]>mean).unsqueeze(1).float()
        return data[index,:],label[index,:]
    def __len__(self,):
        return self.n

In [ ]:
def train(epoch):
#    model.train()
        for i in range(epoch):
            train_loss=0
            for batch_index,(batch_data,batch_label) in enumerate(dataset_loader):
                batch_data=batch_data.to(device)
                batch_label=batch_label.to(device)
                optimizer.zero_grad()
                out=model(batch_data,adjacent,adj2)
                loss=loss_func(out,batch_label)
                loss.backward(retain_graph=True)
                train_loss+=loss.item()
                optimizer.step()
            if i%10==0:
                print("Train Epoch : {},Total Loss is {}".format(i,loss))

In [ ]:
def trans(A):
    size1=A.shape[0]
    A2=np.zeros((size1*size1,size1))
    for i in range(size1):
        sum=0
        for j in range(size1):
            if(A[i][j]==1):
                A2[i*A.shape[0]+j][sum]=1
                sum+=1
    return A2
if __name__=="__main__":
    feature=4
    rand_seed=0
    
    torch.manual_seed(rand_seed)
    mean=torch.randn(feature)
    cov=torch.from_numpy(make_sparse_spd_matrix(feature,alpha=0.95,random_state=0)).float()
    adjacent=(cov!=0).float()
#     adjacent
    max_degree=4 #max(sum(adjacent)).int().item()
    A=[[1,0,0,0],[1,0,1,1],[1,1,0,0],[1,1,1,1]]
    A=np.array(A)
    adj2=trans(A)
    adj2=torch.Tensor(adj2)
    adj2=adj2.to(device)
    data=custom_data(mean,cov,nsample=1000,label_index=2)
    dataset_loader=torch.utils.data.DataLoader(dataset=data,batch_size=4,num_workers=2)
    print("data ready")
    model=GCN_model(feature=feature,max_kernel_size=max_degree,in_channel1=1,out_channel1=3,out_channel2=2,
                    out_channel3=2,hidden1=10,out=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    max_epoch=100
    adjacent=adjacent.to(device)
    loss_func=nn.BCELoss()
    train(max_epoch)

